### This notebook is stale - master is on WML

Git Repo: https://github.ibm.com/wiotp/monitoring-dashboard


In [1]:
import csv
import requests
import collections
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d

In [2]:
#### Code for exporting git defects as csv list
"""
Exports Issues from a specified repository to a CSV file

Uses basic authentication (Github username + password) to retrieve Issues
from a repository that username has access to. Supports Github API v3.
"""

REPO = 'wiotp/monitoring-dashboard'  # format is username/repo
REPO2 = 'wiotp/analytics-service'
REPO_ID = 566611        # retrieved from github via search/repositories
REPO2_ID = 211664

ZEN_WORKSPACE = '5cd44ce4baded506ba8ddd5f'  # retrieved from https://zenhub.ibm.com/p2/repositories/566611/workspaces

# # https://github.ibm.com/wdp-chs/tracker/issues
#AUTH = (GITHUB_USER, GITHUB_PASSWORD)

# 
# #Update your personal API token here
GITHUB_TOKEN = 'ad7e5d3d34e79ac5d06210e74546c36b4bbc86ab'
ZENHUB_TOKEN = '814960f27921de182e39873816e4a54be9cdbb5000baa92d054e773949c1681e5e637c3bd30b54ad'
TRAVIS_TOKEN = '7B8mnG99oqNcHJgKKXMYVQ'

# as per https://zenhub.ibm.com/app/dashboard/tokens
headers = {'Authorization': 'token %s' % GITHUB_TOKEN }


ISSUES_FOR_REPO_URL = 'https://github.ibm.com/api/v3/repos/%s/issues' % REPO
ISSUES_FOR_REPO_URL2 = 'https://github.ibm.com/api/v3/repos/%s/issues' % REPO2

# Update your filter here.  See https://developer.github.com/v3/issues/#list-issues-for-a-repository
# Note that filtering is powerful and there are lots of things available. Also that issues and PRs
# arrive in the same results set
params_payload = {'filter' : 'all', 'state' : 'open', 'type': 'issue' }

# # Update your filter here.  Filter is who owns the issue.  State is open, closed, or all.
params_payload = {'is' : 'issue','filter' : 'all', 'state':'closed' }

credentials = {
  "apikey": "eTgNxIqf0mP7ni5NPSOFkO0lU3nltanbIUZjkecslol_",
  "cos_hmac_keys": {
    "access_key_id": "b1ea25e878064f16a21c2a1e66a9dbaa",
    "secret_access_key": "09dfd480517e95f0e4bd0c50997e546e26183c080ef03dae"
  },
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "Auto-generated for key b1ea25e8-7806-4f16-a21c-2a1e66a9dbaa",
  "iam_apikey_name": "WDP-ML-Manager-137c4937-8276-426f-80a8-c1134f81fabf",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/f8971897be4e1464541021c645cd53fd::serviceid:ServiceId-1a7d5b56-1224-4d03-b20f-6c0888f7e88b",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/f8971897be4e1464541021c645cd53fd:137c4937-8276-426f-80a8-c1134f81fabf::"
}

In [3]:
"""
Exports Issues from a specified repository to a CSV file
Uses basic authentication (Github username + password) or token to retrieve Issues
from a repository that username has access to. Supports Github API v3.
"""

params_payload = {'is' : 'issue', 'state':'all' } # alternative states: all, open, closed


def get_zen_issues(repo_id, zenhub_dict):
    
    zen_url = 'https://zenhub.ibm.com/p2/workspaces/' + ZEN_WORKSPACE + '/repositories/' + str(repo_id) + '/board'
    #print (zen_url)
    kwargs = {
        'headers': {
            'Content-Type': 'application/vnd.github.v3.raw+json',
            'User-Agent': 'Markus zenhub exporter - slightly modified'
        },
        'params': params_payload
    }
    if ZENHUB_TOKEN != '':
        kwargs['headers']['X-Authentication-Token'] = '%s' % ZENHUB_TOKEN
    else:
        kwargs['auth'] = (GITHUB_USER, GITHUB_PASSWORD)

    print ("GET %s" % zen_url)
    resp = requests.get(zen_url, **kwargs)
    print ("  : => %s" % resp.status_code)
    
    
    pipelines = resp.json()['pipelines']
    #print(resp.json())
    for p in pipelines:
        #print (p)
        p_name = p['name']
        for issue in p['issues']:
            number = issue['issue_number']
            epic = issue['is_epic']
            #zenhub_dict[number] = (epic, p_name)
            zenhub_dict[number] = p_name

    
    # import ipdb; ipdb.set_trace()
    if resp.status_code != 200:
        raise Exception(resp.status_code)



def write_issues(response, csvout):
    "output a list of issues to csv"
    print ("  : Writing %s issues" % len(response.json()))

    for issue in response.json():
        # print (issue)
        user = ''
        if issue['user'] is not None:
            user = issue['user']['login']
        
        assignee = ''
        if issue['assignee'] is not None:
            assignee = issue['assignee']['login']
            
        state = issue['state']
        
        assignees_ = []
        for ass in issue['assignees']:
            assignees_.append(ass['login'])
        # print (assignees_)
        
        milestone = issue['milestone']
        if milestone is not None:
            milestone = milestone['title']
            # print (milestone)
        # print (user, assignee, milestone)
        
        labels = issue['labels']
        created_at = issue['created_at']
        if created_at is None:
            created_at_ = '2010-01-01T00:00:00Z'
        created_at = dt.datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%SZ")

        updated_at = issue['updated_at']
        if updated_at is not None:
            updated_at = dt.datetime.strptime(updated_at, "%Y-%m-%dT%H:%M:%SZ")
        
        closed_at = issue['closed_at']
        if closed_at is not None:
            closed_at = dt.datetime.strptime(closed_at, "%Y-%m-%dT%H:%M:%SZ")

        label_string = ''
        componentLabel = ''
        label_list = []
        for label in labels:
              label_list.append(str(label['name']).strip().strip().lstrip().rstrip())
        
        componentLabel = ''
        issueType = 'Issue'
        theme = ''
        blocked = ''
        to_be_closed = ''
        pipeline = ''
        try:
            pipeline = zenhub_dict[issue['number']]
        except Exception:
            pass
        
        for label in label_list:
            
            if label.startswith('Component') or label.startswith('Scrum') or label.startswith('Squad:'):
                label = label.strip()
                componentLabel = label.split(':')[1]
            
            if label.find('Theme') >= 0:
                theme = label
            
            if label.startswith('blocked'):
                blocked = 'YES'
                
            #if label.startswith('')
            
            if label.startswith('Epic'):
                issueType = 'Epic'
            elif label.startswith('bug'):
                issueType = 'Bug'
            elif label.startswith('Enhancement'):
                issueType = 'Enhancement'
                
        # print(label, componentLabel, theme, blocked)
        # 'Title', 'Created', 'Updated', 'Closed', 'Origin', 'Assignee', 'Status', 'Milestone', 'Type', 'Component', 'Theme', 'Blocked', 'Pipeline', 'Labels'))

        csvout.writerow([issue['number'], issue['title'], # issue['body'].encode('utf-8'),
                        created_at, updated_at, closed_at, 
                        user, assignee, state, milestone,
                        issueType, componentLabel,
                        theme, blocked, pipeline, str(label_list)]) # str(label_list).encode('utf-8')])

        
def get_travis_builds(url):
    kwargs = {
        'headers': {
            'Content-Type': 'application/vnd.github.v3.raw+json',
            'User-Agent': 'Padkrish issue exporter - slightly modified'
        },
        'params': params_payload
    }
    if TRAVIS_TOKEN != '':
        kwargs['headers']['Authorization'] = 'token %s' % TRAVIS_TOKEN
    
    #   Travis API 3 doc
    # https://developer.travis-ci.com/resource/builds#Builds
    #
    resp = requests.get('https://api.travis-ci.com/builds')
    
    # TODO - no functionality yet
    

def get_issues(url):
    kwargs = {
        'headers': {
            'Content-Type': 'application/vnd.github.v3.raw+json',
            'User-Agent': 'Padkrish issue exporter - slightly modified'
        },
        'params': params_payload
    }
    if GITHUB_TOKEN != '':
        kwargs['headers']['Authorization'] = 'token %s' % GITHUB_TOKEN
    else:
        kwargs['auth'] = (GITHUB_USER, GITHUB_PASSWORD)

    resp = requests.get("https://github.ibm.com/")
        
    print ("GET %s" % url)
    resp = requests.get(url, **kwargs)
    print ("  : => %s" % resp.status_code)
    
    #print(resp.json())
    
    # import ipdb; ipdb.set_trace()
    if resp.status_code != 200:
        raise Exception(resp.status_code)

    return resp


def next_page(response):
    #more pages? examine the 'link' header returned
    if 'link' in response.headers:
        pages = dict(
            [(rel[6:-1], url[url.index('<')+1:-1]) for url, rel in
                [link.split(';') for link in
                    response.headers['link'].split(',')]])
        # import ipdb; ipdb.set_trace()
        if 'last' in pages and 'next' in pages:
            return pages['next']
    return None


def process(csvout, url=ISSUES_FOR_REPO_URL):                 
    resp = get_issues(url)
    write_issues(resp, csvout)
    next_ = next_page(resp)
    if next_ is not None:
        process(csvout, next_)


csvfilename = '%s-defects-closed-v1.csv' % (REPO.replace('/', '-'))
#s = url.split("/")
#repo_id = REPO_ID
#if s[len(s)-2] != 'monitoring-dashboards':
#    repo_id = REPO2_ID
      
zenhub_dict = {}
get_zen_issues(REPO_ID, zenhub_dict)
    
csvfile = open(csvfilename, 'w', newline='')
csvout = csv.writer(csvfile, quoting=csv.QUOTE_MINIMAL)
csvout.writerow(( 'Title', 'Created', 'Updated', 'Closed', 'Origin', 'Assignee', 'Status', 'Milestone', 'Type', 'Component', 'Theme', 'Blocked', 'Pipeline', 'Labels'))
process(csvout)
# process(csvout, url=ISSUES_FOR_REPO_URL2)
csvfile.close()


GET https://zenhub.ibm.com/p2/workspaces/5cd44ce4baded506ba8ddd5f/repositories/566611/board
  : => 200
GET https://github.ibm.com/api/v3/repos/wiotp/monitoring-dashboard/issues
  : => 200
  : Writing 30 issues
GET https://github.ibm.com/api/v3/repositories/566611/issues?is=issue&state=all&page=2
  : => 200
  : Writing 30 issues
GET https://github.ibm.com/api/v3/repositories/566611/issues?is=issue&state=all&page=3
  : => 200
  : Writing 30 issues
GET https://github.ibm.com/api/v3/repositories/566611/issues?is=issue&state=all&page=4
  : => 200
  : Writing 30 issues
GET https://github.ibm.com/api/v3/repositories/566611/issues?is=issue&state=all&page=5
  : => 200
  : Writing 30 issues
GET https://github.ibm.com/api/v3/repositories/566611/issues?is=issue&state=all&page=6
  : => 200
  : Writing 30 issues
GET https://github.ibm.com/api/v3/repositories/566611/issues?is=issue&state=all&page=7
  : => 200
  : Writing 30 issues
GET https://github.ibm.com/api/v3/repositories/566611/issues?is=issue&

In [4]:
# upload stuff for later use to COS
!s3cmd --access_key "cc04444c99374c9e9589b8f85e931323" --secret_key "1a5062d937b09507a05b521a41b8baf6848c0cd6936e2864" \
--access_token "icny9QB770wS9p67JHCWb6C8ySJEn_3tmfjqct9qQhSY" --host s3.eu.cloud-object-storage.appdomain.cloud --host-bucket=s3.eu.cloud-object-storage.appdomain.cloud \
put wiotp-monitoring-dashboard-defects-closed-v1.csv s3://githubanalyzer-donotdelete-pr-b9xa3kxotzh5in

upload: 'wiotp-monitoring-dashboard-defects-closed-v1.csv' -> 's3://githubanalyzer-donotdelete-pr-b9xa3kxotzh5in/wiotp-monitoring-dashboard-defects-closed-v1.csv'  [1 of 1]
 202741 of 202741   100% in    2s    89.05 kB/s  done


In [5]:
# get it back from COS
!s3cmd --access_key "cc04444c99374c9e9589b8f85e931323" --secret_key "1a5062d937b09507a05b521a41b8baf6848c0cd6936e2864" \
--access_token "icny9QB770wS9p67JHCWb6C8ySJEn_3tmfjqct9qQhSY" --host s3.eu.cloud-object-storage.appdomain.cloud --host-bucket=s3.eu.cloud-object-storage.appdomain.cloud \
get s3://githubanalyzer-donotdelete-pr-b9xa3kxotzh5in/wiotp-monitoring-dashboard-defects-closed-v1.csv wiotp-monitoring-dashboard-defects.csv

download: 's3://githubanalyzer-donotdelete-pr-b9xa3kxotzh5in/wiotp-monitoring-dashboard-defects-closed-v1.csv' -> 'wiotp-monitoring-dashboard-defects.csv'  [1 of 1]
 202741 of 202741   100% in    1s   165.95 kB/s  done


In [6]:
defects_df = pd.read_csv('./wiotp-monitoring-dashboard-defects.csv') # parse_dates=['Created'])

## Remove the encoding prefix string
df=defects_df.replace('b\'','',regex=True).astype(object)
df=df.replace('b\"','',regex=True).astype(object)
df['Created'] = pd.to_datetime(df['Created'])
df['Updated'] = pd.to_datetime(df['Updated'])
df['Closed'] = pd.to_datetime(df['Closed'])

df = df[df['Pipeline'] != 'To Be Closed']
df.head(3)

,Title,Created,Updated,Closed,Origin,Assignee,Status,Milestone,Type,Component,Theme,Blocked,Pipeline,Labels
921,Session times out after getting timeouts on en...,2020-04-03 16:49:07,2020-04-03 16:49:07,NaT,jhaa,rsiva,open,NaN,Bug,Monitoring,NaN,NaN,New Issues,"['Squad: Monitoring', 'bug']"
920,Clicking the numeric or string value of a Prop...,2020-04-03 16:37:55,2020-04-03 16:38:06,NaT,scottsd,Joel-Armendariz,open,April 2020 Sprint 13,Bug,Monitoring,NaN,NaN,New Issues,"['Squad: Monitoring', 'bug']"
919,Database connection exhausted for Omnicell tenant,2020-04-03 16:29:46,2020-04-03 17:17:31,2020-04-03 17:17:31,saurabh-gupta-ibm,NaN,closed,NaN,Issue,NaN,NaN,NaN,NaN,[]


In [8]:
#invent severities
df['severity'] = np.random.choice([1,2,3,4], df.index.size, p = [0.1,0.4,0.3,0.2])
df.head(3)

,Title,Created,Updated,Closed,Origin,Assignee,Status,Milestone,Type,Component,Theme,Blocked,Pipeline,Labels,severity
921,Session times out after getting timeouts on en...,2020-04-03 16:49:07,2020-04-03 16:49:07,NaT,jhaa,rsiva,open,NaN,Bug,Monitoring,NaN,NaN,New Issues,"['Squad: Monitoring', 'bug']",2
920,Clicking the numeric or string value of a Prop...,2020-04-03 16:37:55,2020-04-03 16:38:06,NaT,scottsd,Joel-Armendariz,open,April 2020 Sprint 13,Bug,Monitoring,NaN,NaN,New Issues,"['Squad: Monitoring', 'bug']",2
919,Database connection exhausted for Omnicell tenant,2020-04-03 16:29:46,2020-04-03 17:17:31,2020-04-03 17:17:31,saurabh-gupta-ibm,NaN,closed,NaN,Issue,NaN,NaN,NaN,NaN,[],2


In [9]:
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

w = widgets.IntSlider()
display(w)

IntSlider(value=0)

In [17]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


def f(severity):
    dfs = df
    if severity > 0:
        dfs = df[df['severity'] == severity]
    cnt = dfs.groupby('Type')['Title'].count()
    labels = cnt.index
    sizes = cnt.values
    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, explode=(0.04, 0, 0), labels=labels, autopct='%1.1f%%',
            shadow=True, startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    ax1.set_title('Monitoring issues by type')

#plt.show()
interact(f, severity=[('sev1', 1), ('sev2', 2), ('sev3',3), ('sev4',4), ('all', 0)]);

interactive(children=(Dropdown(description='severity', options=(('sev1', 1), ('sev2', 2), ('sev3', 3), ('sev4'…

### Velocities

- Overall Velocity
- Defect Velocity (**implemented** - starting mid 2019)
- Customer satisfaction (**no data** not included as we have no data)
- Defect breakdown by severity (**no data** we have no severities in our defects)
- Value per sprint (**no data** value of epics or stories is not known)
- Percentage of test case coverage (**no data** - we do not track that)
- Customer Support Requests (**no data*)
- Trends (let's define what trends we need)
- NPS (**no data**)
- Build failures (need to get Travis data)
- Deployment failures (Jenkins ?)
- Technical Debts (**no data** we need a label)

In [57]:
# defect velocity since mid 2019
StartWith = '2019-06-30'


df_velocity = df.copy()
df_velocity = df_velocity[df_velocity['Status'] == 'closed']
df_velocity = df_velocity[df_velocity['Created'] > StartWith]
df_velocity['Age'] = df_velocity['Closed'] - df_velocity['Created'] + pd.Timestamp('2020-01-01')
df_velocity = df_velocity[['severity', 'Age']]


def f(severity):
    df_vel = df_velocity.copy()
    if severity != 0:
        df_vel = df_vel[df_vel['severity'] == severity]
    df_vel = df_vel[['Age']]
    #df_vel.groupby([df_vel["Age"].dt.year,
    #              df_vel["Age"].dt.month, df_vel["Age"].dt.week]).count().plot(figsize=(10,8), kind="bar", title='Defect velocity by weeks')
    df_vel.groupby(df_vel['Age'].dt.week).count().plot(figsize=(7,5), kind='bar')
    ax1 = plt.axes()
    x_axis = ax1.axes.get_xaxis()
    # x_axis.set_visible(False)

interact(f, severity=[('sev1', 1), ('sev2', 2), ('sev3',3), ('sev4',4), ('all', 0)]);

interactive(children=(Dropdown(description='severity', options=(('sev1', 1), ('sev2', 2), ('sev3', 3), ('sev4'…

In [16]:
### Initiatives, Epics, Stories

- Epics per Initiatives
- Initiative completion by percentage
- Percentage of Epics completed
- Percentage of stories completed by sprint vs what was committed
- Defects per story
- End to end start to completion average

SyntaxError: invalid syntax (<ipython-input-16-a37208589025>, line 3)